In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

Fri May 28 08:24:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 443Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 6.58Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 618Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:55, 8.92Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 263Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 8.91Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 8.11Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1VE2L7AuuR2EkLBZiX9P8ybVTBjHsSsKT'
output = 'my_archive.tar'
gdown.download(url, output, quiet=False)
!unzip -q 'my_archive.tar'


Downloading...
From: https://drive.google.com/uc?id=1VE2L7AuuR2EkLBZiX9P8ybVTBjHsSsKT
To: /content/my_archive.tar
2.34MB [00:00, 114MB/s]


In [ ]:
import pandas as pd
import glob

path = r'results/job_hero/' # use your path
file_list = glob.glob(path + "/*.txt")


import fileinput

with open('cover_letters.txt', 'w') as file:
    input_lines = fileinput.input(file_list)
    file.writelines('\n'.join(input_lines))

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory


In [ ]:
file_name='cover_letters.txt'

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
??gpt2.finetune

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:03<00:00,  3.12s/it]


dataset has 581723 tokens
Training...
[10 | 28.64] loss=2.65 avg=2.65
[20 | 50.93] loss=2.48 avg=2.56
[30 | 73.67] loss=2.44 avg=2.52
[40 | 97.10] loss=2.33 avg=2.47
[50 | 120.97] loss=2.32 avg=2.44
[60 | 144.65] loss=2.09 avg=2.38
[70 | 168.24] loss=2.01 avg=2.33
[80 | 191.85] loss=1.99 avg=2.28
[90 | 215.54] loss=1.93 avg=2.24
[100 | 239.23] loss=1.99 avg=2.22
[110 | 262.97] loss=1.96 avg=2.19
[120 | 286.72] loss=2.00 avg=2.18
[130 | 310.47] loss=1.85 avg=2.15
[140 | 334.20] loss=1.90 avg=2.13
[150 | 357.87] loss=2.01 avg=2.12
[160 | 381.58] loss=1.93 avg=2.11
[170 | 405.22] loss=1.83 avg=2.09
[180 | 428.84] loss=1.83 avg=2.08
[190 | 452.55] loss=1.79 avg=2.06
[200 | 476.18] loss=1.93 avg=2.05
======== SAMPLE 1 ========
 upgrades and enhancements in order to help drive growth in both client base and profits.

 Increasing sales by 43% through seamless enhancements to the sales process.

 Utilizing persuasive communication talents to drive sales and propel increased profits.

With my c

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
import tensorflow as tf
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=700,
              temperature=0.7,
              prefix="I am writing to you to submit my resume for consideration as your next Data Specialist.",
              truncate="<Sincerly>",
              nsamples=5,
              batch_size=5
              )

I am writing to you to submit my resume for consideration as your next Data Specialist. As an analytical and bottom-line-driven professional with more than eight years of experience leading data analysis and process improvement programs, I feel confident of my ability to generate superior results for your organization.

My background includes managing project management software and systems, managing team deployments, and generating detailed reports for senior management teams. With an inherent passion for excellence, I will make a significant impact on your organization.

I invite you to consider the following highlights of my qualifications:

 Overseeing multimillion-dollar software projects for clients in diverse sectors including healthcare, manufacturing, and consumer products.

 Developing, maintaining, and continually updating internal staff to drive increased productivity and outperform goals.

 Implementing a new system to capture and track data across various departments and 

In [ ]:
gpt2.generate(sess,
              length=1000,
              temperature=0.7,
              prefix="Dear Ms. Mustermann, with this letter and attached resume, I would like to express my sincere interest in the vacancy for ‘UI/UX Designer’.",
              truncate="Sincerely",
              nsamples=5,
              batch_size=5,
              include_prefix=False
              )

 As an accomplished and customer-centric professional with more than 13 years of experience designing and implementing user-friendly interface solutions to efficiently and successfully meet the needs of both end users and designers, I feel confident in my ability to significantly contribute to the success of your company.

My experience includes designing and implementing a broad range of user-friendly design solutions to drive market share, performance, and customer engagement. With my professional achievements, combined with my technical and customer service capabilities, I am confident that my talents and abilities will significantly benefit Yougalers.

The following achievements demonstrate my qualification for this position:

 Designing user interfaces to ensure seamless interface solutions and corresponding solutions to achieve customer needs and objectives.

 Spearheading the development and implementation of software features—including support, troubleshooting, integration, and

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1